In [ ]:
import glob
import cv2 as cv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.io import imread, imshow

from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D , Input
from keras.layers import Activation, Dropout, Flatten, Dense
from keras.optimizers import SGD, Adam
from keras.models import Sequential
from keras.layers import BatchNormalization, Dropout
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from sklearn.utils import class_weight
from sklearn import metrics
from keras import regularizers
from keras.callbacks import ModelCheckpoint

In [ ]:
data = np.load('../input/cnn-lbp/Train_LBP_v2.npz')

In [ ]:
test_data = np.load('../input/cnn-lbp/Test_LBP_v2.npz')

In [ ]:
X_test = test_data['features_lbp']
Y_test = test_data['labels_lbp']

In [ ]:
data.files

In [ ]:
features = data['features_lbp']
labels = data['labels_lbp']

In [ ]:
img = np.reshape(features[9],(150,150,3))

In [ ]:
plt.imshow(img)
plt.title('LBP Image')
plt.axis('off')

In [ ]:
#X_train, X_val, Y_train, Y_val = train_test_split(features, labels, test_size=0.20, random_state=42)

In [ ]:
class_weights = class_weight.compute_class_weight('balanced',np.unique(labels),labels)

print(class_weights)

In [ ]:
def build_cnn():
    
    model = Sequential()
    model.add(Conv2D(32, (3, 3), input_shape=(150,150,3) , activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(BatchNormalization(axis = -1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(BatchNormalization(axis = -1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(256, (3, 3), activation='relu'))
    model.add(BatchNormalization(axis = -1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    
    model.add(Conv2D(512, (3, 3), activation='relu'))
    model.add(BatchNormalization(axis = -1))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))

    model.add(Flatten())
    model.add(Dense(256 ,  activation='relu'))
    model.add(BatchNormalization(axis = -1))
    model.add(Dropout(0.2))
    model.add(Dense(2 , activation ='softmax'))

    return model

In [ ]:
model = build_cnn()
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy']) 

In [ ]:
model.summary()

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=7, random_state=None)

cvscores = []
acc = []
vacc = []
loss = []
vloss = []
i = 0
for train_index, val_index in skf.split(features,labels):
    i = i + 1
    X_train, X_val = features[train_index], features[val_index]
    Y_train, Y_val = labels[train_index], labels[val_index]
    name = 'model_LBP_v' + str(i) + '.hdf5'
    mcp = ModelCheckpoint(filepath=name, monitor="val_accuracy", save_best_only=True, save_weights_only=False)
    
    print('\n')
    print('Start Training Fold_' + str(i)+'--------------------------------------')
    #print('\n')
    hist = model.fit(X_train,Y_train, validation_data = (X_val,Y_val) , 
                         epochs = 20,verbose=1,class_weight = class_weights,callbacks=[mcp])
    print('\n')
    scores = model.evaluate(X_val,Y_val, verbose=1)
    print('\n-------------Model Evaluation Results '+ 'Fold_' + str(i) + '-----------')
    print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
    print('\n------------------------------------------------------------------------------------------------------------')
    cvscores.append(scores[1] * 100)
    acc.append(hist.history['accuracy'])
    vacc.append(hist.history['val_accuracy'])
    loss.append(hist.history['loss'])
    vloss.append(hist.history['val_loss'])

print("%.2f%% (+/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))

# Predictions on Test Data

In [ ]:
X_test.shape

In [ ]:
# Cross-Validation Predictions
test_results = model.evaluate(X_test,Y_test)
y_test_pred = model.predict_classes(X_test)

print('\nCV_Results_Overall Model Performance')
print(test_results)

## Evaluation of each fold

In [ ]:
fold_results = []

In [ ]:
model.load_weights('../input/weights/model_LBP_v1.hdf5')
fold1_results = model.evaluate(X_test,Y_test)

print('\n---CV_Results_Fold_1---')
print(fold1_results)
fold_results.append(fold1_results)

In [ ]:
model.load_weights('../input/weights/model_LBP_v2.hdf5')
fold2_results = model.evaluate(X_test,Y_test)

print('\n---CV_Results_Fold_2---')
print(fold2_results)
fold_results.append(fold2_results)

In [ ]:
model.load_weights('../input/weights/model_LBP_v3.hdf5')
fold3_results = model.evaluate(X_test,Y_test)

print('\n---CV_Results_Fold_3---')
print(fold3_results)
fold_results.append(fold3_results)

In [ ]:
model.load_weights('../input/weights/model_LBP_v4.hdf5')
fold4_results = model.evaluate(X_test,Y_test)

print('\n---CV_Results_Fold_4---')
print(fold4_results)
fold_results.append(fold4_results)

In [ ]:
model.load_weights('../input/weights/model_LBP_v5.hdf5')
fold5_results = model.evaluate(X_test,Y_test)

print('\n---CV_Results_Fold_5---')
print(fold5_results)
fold_results.append(fold5_results)

In [ ]:
model.load_weights('../input/weights/model_LBP_v6.hdf5')
fold6_results = model.evaluate(X_test,Y_test)

print('\n---CV_Results_Fold_6---')
print(fold6_results)
fold_results.append(fold6_results)

In [ ]:
model.load_weights('../input/weights/model_LBP_v7.hdf5')
fold7_results = model.evaluate(X_test,Y_test)

print('\n---CV_Results_Fold_7---')
print(fold7_results)
fold_results.append(fold7_results)

In [ ]:
fold_results = np.array(fold_results)

In [ ]:
overall_loss = fold_results[:,0]
overall_accu = fold_results[:,1]

In [ ]:
print("\n\nTest Data Accuracy : %.2f%% (+/- %.2f%%)" % (np.mean(overall_accu)*100, np.std(overall_accu)*100))

# Best Model Results

In [ ]:
model.load_weights('../input/weights/model_LBP_v6.hdf5')

In [ ]:
y_pred = model.predict_classes(X_test)

print('Best CV Fold Accuracy %.2f%%' % (metrics.accuracy_score(Y_test, y_pred)*100))

In [ ]:
print('Model Accuracy %.2f%%' % (metrics.accuracy_score(Y_test, y_test_pred)*100))

# Visualise Results

## Train Validation Accuracy

In [ ]:
acc_1 = np.hstack((acc[0],acc[1],acc[2],acc[3],acc[4],acc[5],acc[6]))
vacc_1 = np.hstack((vacc[0],vacc[1],vacc[2],vacc[3],vacc[4],vacc[5],vacc[6]))
loss_1 = np.hstack((loss[0],loss[1],loss[2],loss[3],loss[4],loss[5],loss[6]))
vloss_1 = np.hstack((vloss[0],vloss[1],vloss[2],vloss[3],vloss[4],vloss[5],vloss[6]))

In [ ]:
def plot_model_history(acc,vacc,loss,vloss):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    axs[0].plot(range(1,len(acc)+1),acc)
    axs[0].plot(range(1,len(vacc)+1),vacc)
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    axs[0].set_xticks(np.arange(1,len(acc)+1),len(acc)/10)
    axs[0].legend(['train', 'val'], loc='best')
    axs[1].plot(range(1,len(loss)+1),loss)
    axs[1].plot(range(1,len(vloss)+1),vloss)
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    axs[1].set_xticks(np.arange(1,len(loss)+1),len(loss)/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

In [ ]:
plot_model_history(acc_1,vacc_1,loss_1,vloss_1)

# Confusion Matrix

In [ ]:
def plot_confusion_matrix(cm,target_names,title='Confusion matrix',cmap=None,normalize=True):
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap('Blues')

    plt.figure(figsize=(6, 4))
    plt.grid(b=False)
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title,fontsize=12,weight='bold')
    #plt.legend('off')
    #plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=0,fontsize=12,weight='bold')
        plt.yticks(tick_marks, target_names,fontsize=12,weight='bold')
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(j, i, "{:0.4f}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",fontsize=12,weight='bold')
        else:
            plt.text(j, i, "{:,}".format(cm[i, j]),
                     horizontalalignment="center",
                     color="white" if cm[i, j] > thresh else "black",fontsize=12,weight='bold')
    plt.tight_layout()
    plt.ylabel('True label',fontsize=12,weight='bold')
    plt.xlabel('\n Predicted label \n Test accuracy={:0.2f}%; Misclass={:0.2f}%'.format(accuracy*100, misclass*100),fontsize=12,weight='bold')
    #plt.get_legend().remove()
    
    plt.show()

In [ ]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test,y_pred)
plot_confusion_matrix(cm = cm,normalize= False,cmap ='binary',target_names = ['nocancer', 'cancer'],
                      title= '')

In [ ]:
tn, fp, fn, tp = confusion_matrix(Y_test,y_pred).ravel()
print(tn, fp, fn, tp)

In [ ]:
print("Sensitivity %.2f%%" % ((tp / (tp + fn))*100))

In [ ]:
print("Specificity %.2f%%" % ((tn / (tn + fp))*100))

# Classification Report

In [ ]:
from sklearn.metrics import classification_report

# Generate a classification report
report = classification_report(Y_test, y_pred, target_names=['nocancer','cancer'])

print(report)

In [ ]:
print("%.2f%%" % (metrics.f1_score(Y_test, y_pred,average='weighted')*100))

# Saving Train / Validation Data

In [ ]:
LBP_result = pd.DataFrame()

In [ ]:
LBP_result['lbp_acc'] = acc_1
LBP_result['lbp_vacc'] = vacc_1
LBP_result['lbp_loss'] = loss_1
LBP_result['lbp_vloss'] = vloss_1

In [ ]:
LBP_result.head()

In [ ]:
LBP_result.to_csv('lbp_results.csv', sep='\t', encoding='utf-8')